# Feature Construction & Feature Splitting

`Feature construction` and `Feature splitting` are techniques used to enrich the feature space by creating
more informative representations from existing data. These methods often help models capture patterns
that are not directly visible in the raw features.

---

## Feature Construction

Feature construction involves **creating new features** by combining or transforming existing ones.

### Common Examples
- Ratios (e.g., price per unit, income per household)  
- Sums and differences  
- Interaction features (e.g., \( x_1 \times x_2 \))  
- Polynomial features  

### Why Feature Construction is Useful

- Introduces meaningful patterns the model can learn  
- Captures non-linear relationships  
- Leverages domain knowledge  
- Can significantly improve model performance  

### When to Use

- Domain knowledge suggests hidden relationships  
- Raw features alone are insufficient  
- Linear or low-complexity models need more expressive power  

### Models That Benefit

- Linear models  
- Tree-based models  
- Boosting algorithms  

---

## Feature Splitting

Feature splitting breaks a single feature into **multiple meaningful components**.

### Common Examples
- Datetime → year, month, day, weekday  
- Text → tokens, n-grams  
- Categorical strings → structured sub-parts  

### Why Feature Splitting is Useful

- Exposes multiple signals from a single feature  
- Improves interpretability  
- Enables models to learn finer-grained patterns  

### When to Use

- Features contain multiple embedded concepts  
- Working with temporal, categorical, or text-like data  
- Raw feature is too coarse  

### Models That Benefit

- Tree-based models  
- NLP models  
- Time-series algorithms  

---

## Best Practices

- Apply feature construction and splitting after basic cleaning  
- Avoid creating redundant or highly correlated features  
- Validate new features using cross-validation  
- Use pipelines to ensure reproducibility  

---

## Summary

Feature construction and feature splitting are powerful feature engineering techniques that transform
raw data into richer representations. When applied thoughtfully, they can significantly enhance
model performance, interpretability, and robustness.


In [1]:
%%capture
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn


In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

import seaborn as sns

In [3]:
df = pd.read_csv('Titanic.csv')[['Age','Pclass','SibSp','Parch','Survived']]
df.head()


,Age,Pclass,SibSp,Parch,Survived
0,22.0,3,1,0,0
1,38.0,1,1,0,1
2,26.0,3,0,0,1
3,35.0,1,1,0,1
4,35.0,3,0,0,0


In [4]:
df.dropna(inplace=True)
df.head()

,Age,Pclass,SibSp,Parch,Survived
0,22.0,3,1,0,0
1,38.0,1,1,0,1
2,26.0,3,0,0,1
3,35.0,1,1,0,1
4,35.0,3,0,0,0


In [5]:
X = df.iloc[:,0:4]
y = df.iloc[:,-1]
X.head()

,Age,Pclass,SibSp,Parch
0,22.0,3,1,0
1,38.0,1,1,0
2,26.0,3,0,0
3,35.0,1,1,0
4,35.0,3,0,0


In [6]:
np.mean(cross_val_score(LogisticRegression(),X,y,scoring='accuracy',cv=20))

np.float64(0.6933333333333332)

### Applying Feature Construction

In [7]:
X['Family_size'] = X['SibSp'] + X['Parch'] + 1
X.head()

,Age,Pclass,SibSp,Parch,Family_size
0,22.0,3,1,0,2
1,38.0,1,1,0,2
2,26.0,3,0,0,1
3,35.0,1,1,0,2
4,35.0,3,0,0,1


In [8]:
def myfunc(num):
    if num == 1:
        #alone
        return 0
    elif num >1 and num <=4:
        # small family
        return 1
    else:
        # large family
        return 2

In [9]:
myfunc(4)


1

In [10]:
X['Family_type'] = X['Family_size'].apply(myfunc)
X.head()

,Age,Pclass,SibSp,Parch,Family_size,Family_type
0,22.0,3,1,0,2,1
1,38.0,1,1,0,2,1
2,26.0,3,0,0,1,0
3,35.0,1,1,0,2,1
4,35.0,3,0,0,1,0


In [11]:
X.drop(columns=['SibSp','Parch','Family_size'],inplace=True)
X.head()

,Age,Pclass,Family_type
0,22.0,3,1
1,38.0,1,1
2,26.0,3,0
3,35.0,1,1
4,35.0,3,0


In [12]:
np.mean(cross_val_score(LogisticRegression(),X,y,scoring='accuracy',cv=20))

np.float64(0.7003174603174602)

### Feature Splitting

In [13]:
df = pd.read_csv('Titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
df['Name']

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

In [15]:
df['Title'] = df['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]

In [16]:
df['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]

0        Mr
1       Mrs
2      Miss
3       Mrs
4        Mr
       ... 
886     Rev
887    Miss
888    Miss
889      Mr
890      Mr
Name: 0, Length: 891, dtype: object

In [17]:
df[['Title','Name']]

,Title,Name
0,Mr,"Braund, Mr. Owen Harris"
1,Mrs,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,Miss,"Heikkinen, Miss. Laina"
3,Mrs,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,Mr,"Allen, Mr. William Henry"
...,...,...
886,Rev,"Montvila, Rev. Juozas"
887,Miss,"Graham, Miss. Margaret Edith"
888,Miss,"Johnston, Miss. Catherine Helen ""Carrie"""
889,Mr,"Behr, Mr. Karl Howell"


In [21]:
df.groupby('Title')['Survived'].mean().sort_values(ascending=False).reset_index()


,Title,Survived
0,Lady,1.000000
1,Ms,1.000000
2,Sir,1.000000
3,Mme,1.000000
4,the Countess,1.000000
5,Mlle,1.000000
6,Mrs,0.792000
7,Miss,0.697802
8,Master,0.575000
9,Major,0.500000


In [22]:
df['Is_Married'] = 0
df['Is_Married'].loc[df['Title'] == 'Mrs'] = 1

C:\Users\Msi Laptop\AppData\Local\Temp\ipykernel_20092\2254989826.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Is_Married'].loc[df['Title'] == 'Mrs'] = 1
C:\Users\Msi Laptop\AppData\Local\Temp\ipykernel_20092\2254989826.py:2: Sett

In [23]:
df['Is_Married']

0      0
1      1
2      0
3      1
4      0
      ..
886    0
887    0
888    0
889    0
890    0
Name: Is_Married, Length: 891, dtype: int64